In [ ]:
#Install the proper libraries

In [1]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
!pip install PyPDF2
!pip install faiss-gpu
import PyPDF2
import chromadb
import faiss
hf_api_key = os.environ.get('HF_API_KEY')
#hub_api_key = os.environ.get('HUGGINGFACEHUB_API_TOKEN')
nb_key =   os.environ.get('HF_NB_Token')
from huggingface_hub import notebook_login #needed this to access gated repos on HF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.5 MB/s eta 0:00:0000:0100:01


In [2]:
notebook_login(hf_api_key)

In [ ]:
#Need to load the PDF document that will then be chunked and stored in the local vectorDB

In [ ]:
#Need to be aware of constraints when it comes to chunking prior to vector embedding
#1. Source document should contain the desired information. RAG will not be helpful
#if the document doesn't provide the requisite info needed to answer your questions
#2. Once the doc is split into chunks, the chunks should be small enough to fit into the context 
#window of your chosen model
#3. Chunks should be small enough to fit into the embeddings model. often times these embedding models
#have their token limits

In [ ]:
#Need to load the PDF document that will then be chunked 
#this implementation cycles through all the pages and concatenates them into the object
#"output"

In [3]:
import PyPDF2
pdfFileObject = open("growth_loops_product_school.pdf", 'rb')
pdfReader = PyPDF2.PdfReader(pdfFileObject)
count = pdfReader.pages
full_doc = ''
for i in range(len(count)):
    page = pdfReader.pages[i]
    full_doc += page.extract_text()


In [4]:
#the output of this "full_doc" is just one big string and is not a 'Document'
#object which is required for other langchain calls. Define a function that creates document objects from the chunks

In [4]:
def generate_tokens(s):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
  splits = text_splitter.split_text(s)

  return text_splitter.create_documents(splits) #th

In [5]:
#chunk the doc

In [6]:
docs_after_split = generate_tokens(full_doc)

In [7]:
#encode the chunks now

In [8]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="mixedbread-ai/mxbai-embed-large-v1",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
   #model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
    model_kwargs={'device':'cpu'}, 
    encode_kwargs={'normalize_embeddings': True}
    
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs-us-1.hf.co', port=443): Max retries exceeded with url: /repos/73/9c/739c7bd39907498a59f67f6e49700d5018e3916870557a0db41ec2cbe244fd34/36bfa45da00eb762ef9feebe4cff315ec779efadd08da11846bef5ba5b59b8f8?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1731184835&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTE4NDgzNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzczLzljLzczOWM3YmQzOTkwNzQ5OGE1OWY2N2Y2ZTQ5NzAwZDUwMThlMzkxNjg3MDU1N2EwZGI0MWVjMmNiZTI0NGZkMzQvMzZiZmE0NWRhMDBlYjc2MmVmOWZlZWJlNGNmZjMxNWVjNzc5ZWZhZGQwOGRhMTE4NDZiZWY1YmE1YjU5YjhmOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=R1N0EusJ2j7QjcqqfOak-Fzs1v9X~1FKwLuEW-bc5~M2ekwQn-N6i4wDK3qev7ZyM7jTi0hL~f5yM3V2TtHrhF-jcytYL35Tq-lWAdxuQjt-KfHq20W3r-12q8vdGdKKMd3w~YLZTqBfZc~2a2Y9idQdwrSV3auMa2DIbmmabpaVlMXcO0WdXFZayisAlrDZlBezJkwX

In [8]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [-0.0191627   0.02619416  0.00914849 ... -0.03005453  0.03546663
  0.01888874]
Size of the embedding:  (1024,)


In [9]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

In [10]:
query = """What are some companies that use a referral loop"""  
         # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query)
print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
print(relevant_documents[0].page_content)

There are 4 documents retrieved which are relevant to the query. Display the first one:

side of the marketplace attracts the other, creating a balance that sustain s growth.  
Real-Life Examples of Product Loops Used Well  
Dropbox – Referral Loop  
Dropbox is a classic example of a referral loop in action. When they first launched, they 
offered users extra storage space for every friend they invited to join. New users would then 
invite others, because who doesn’t want more free space?


In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM
PATH = 'growth_loops_RAG/Mistral_7B/config.json'
tokenizer = AutoTokenizer.from_pretrained(PATH)

OSError: Incorrect path_or_model_id: 'growth_loops_RAG/Mistral_7B/config.json'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir = "C:/Users/BuCu154/Desktop")
llm_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", cache_dir = "C:/Users/BuCu154/Desktop")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
#llm.invoke(query)

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Max retries exceeded with url: /repos/78/c6/78c6450f2e4f8ec5f8c6c5ac2c4ecff0e6c9dadc650fa7daf2d5bc0323dc99f6/dadfd56d766715c61d2ef780a525ab43b8e6da4de6865bda3d95fdef5e134055?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tokenizer.model%3B+filename%3D%22tokenizer.model%22%3B&Expires=1731185194&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTE4NTE5NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy83OC9jNi83OGM2NDUwZjJlNGY4ZWM1ZjhjNmM1YWMyYzRlY2ZmMGU2YzlkYWRjNjUwZmE3ZGFmMmQ1YmMwMzIzZGM5OWY2L2RhZGZkNTZkNzY2NzE1YzYxZDJlZjc4MGE1MjVhYjQzYjhlNmRhNGRlNjg2NWJkYTNkOTVmZGVmNWUxMzQwNTU~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=G~l0oDP8lYt4Q8zkigXMQ-GOw4NziSEsD-9pTfo5fr8XSRqFrDO-Dryx5gxCZHyQKLLMAp1LFLcqAASOGCl~7~jpWTItCDBPMQtTP37IL8B6sLqT2~bARjyWdZK~KcKKHeCJIefwwfPOHuTteEmkkZ43VGcgCb10on5cRk69E07sP1Ld4IPJQnMpM51LDXMFFl~VJKWYNFNOkt6yGo2-L

In [ ]:
prompt_template = """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])